In [1]:
# LoRA lora ile fine tune ederken
## 50x50 bir matrisimiz varsa 2500 parametremiz var.
### ve biz 200 parametreyi değiştirmiş oluyoruz.
#### bu da az parametre değişikliğiyle çok büyük değişiklikler elde etmektir.

In [2]:
!pip install -q peft==0.8.2
!pip install -q datasets==2.16.1

In [3]:
# Transformers kütühanesini kullanarak model ve tokenize import etme işlemi.
# Bir dil modeli mimarisi vermeden, bunu model kendi kendine anlaması için AutoNodelForCasualLM modeli.
from transformers import AutoModelForCausalLM, AutoTokenizer


model_name = "bigscience/bloomz-560m"

# bloom çok küçük ve akıllı bir model. Modelimiz foundation_model.

tokenizer = AutoTokenizer.from_pretrained(model_name)
foundation_model = AutoModelForCausalLM.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [4]:
# elimizdeki modeli fine_tuning etmeden deneyelim. Bizim için
# bu görevi yapacak bir fonksiyon

def get_output(model, inputs, max_new_tokens=100):
  outputs = model.generate(
      input_ids = inputs["input_ids"],
      attention_mask= inputs["attention_mask"],
      max_new_tokens = max_new_tokens,
      repetition_penalty = 1.5, # tekrarlardan kaçınmak için
      early_stopping = True, # max uzunluğa ulaştığı zaman model ürteimi durdursun
      eos_token_id = tokenizer.eos_token_id
  )
  return outputs

In [5]:
input_sentence = tokenizer("ı want you to be a motivational coach.   ",return_tensors="pt")
foundational_outputs_sentence = get_output(foundation_model, input_sentence,max_new_tokens=100)

print(tokenizer.batch_decode(foundational_outputs_sentence, skip_special_tokens=True))

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:535: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


['ı want you to be a motivational coach.    I know that this is not your job.']


In [6]:
# Başarısız. şimdi eğitim için hazırlıklar.

In [7]:
## huggingface de chatgpt promptlarını içeren veri seti

from datasets import load_dataset
dataset = "fka/awesome-chatgpt-prompts"

data = load_dataset(dataset)
data


DatasetDict({
    train: Dataset({
        features: ['act', 'prompt'],
        num_rows: 153
    })
})

In [8]:
data = data.map(lambda samples: tokenizer(samples["prompt"]), batched=True)

In [9]:
# tokenizerdan geçirdiğim prompt sütunundan sonra input_ids ve attention_mask sütunları oluştu.

data

DatasetDict({
    train: Dataset({
        features: ['act', 'prompt', 'input_ids', 'attention_mask'],
        num_rows: 153
    })
})

In [10]:
train_sample = data["train"].select(range(50))   # bir liste üzerinde çalışlıldığı için select uygun.
train_sample = train_sample.remove_columns('act')

display(train_sample)

Dataset({
    features: ['prompt', 'input_ids', 'attention_mask'],
    num_rows: 50
})

In [11]:
print(train_sample[:1])

{'prompt': ['I want you to act as a linux terminal. I will type commands and you will reply with what the terminal should show. I want you to only reply with the terminal output inside one unique code block, and nothing else. do not write explanations. do not type commands unless I instruct you to do so. when i need to tell you something in english, i will do so by putting text inside curly brackets {like this}. my first command is pwd'], 'input_ids': [[44, 4026, 1152, 427, 1769, 661, 267, 104105, 28434, 17, 473, 2152, 4105, 49123, 530, 1152, 2152, 57502, 1002, 3595, 368, 28434, 3403, 6460, 17, 473, 4026, 1152, 427, 3804, 57502, 1002, 368, 28434, 10014, 14652, 2592, 19826, 4400, 10973, 15, 530, 16915, 4384, 17, 727, 1130, 11602, 184637, 17, 727, 1130, 4105, 49123, 35262, 473, 32247, 1152, 427, 727, 1427, 17, 3262, 707, 3423, 427, 13485, 1152, 7747, 361, 170205, 15, 707, 2152, 727, 1427, 1331, 55385, 5484, 14652, 6291, 999, 117805, 731, 29726, 1119, 96, 17, 2670, 3968, 9361, 632, 269, 4

In [12]:
# Fine Tuning

import peft
from peft import LoraConfig, get_peft_model, PeftModel

lora_config = LoraConfig(
    r=4, # daha büyük r eğitim için daha büyük train demek
    lora_alpha=1, #ağırlık matrisi için. genelde 1 olarak kalır
    target_modules=["query_key_value"], # genellikle metin anlamını belirleyen veya dil bilgisi yapılarını öğrenen dönüşümsel katmanları hedefleyebilirsiniz. Bu katmanlar, genellikle dil modelinin temelini oluşturur ve büyük ağırlık matrislerine sahiptir. LoRA kullanarak, bu tür katmanların boyutunu azaltarak modelin daha hafif ve hızlı çalışır.
    lora_dropout=0.05,
    bias="lora_only",
    task_type="CASUAL_LM"
)

"""
En önemli parametre r dır. Ne kadar parametrenin bu modelde eğitileceğine karar verir.
çok büyük r yaparsak mmodel girdi ve çıktı arasındaki kompleks ilişkiyii daha iyi kavrar.
task_type şudur. modelin amacı ne? bu modelin amacı text generation..

"""

'\nEn önemli parametre r dır. Ne kadar parametrenin bu modelde eğitileceğine karar verir.\nçok büyük r yaparsak mmodel girdi ve çıktı arasındaki kompleks ilişkiyii daha iyi kavrar.\ntask_type şudur. modelin amacı ne? bu modelin amacı text generation.. \n\n'

In [13]:
## PEFT model

peft_model = get_peft_model(foundation_model, lora_config)
print(peft_model.print_trainable_parameters())

# modelin eğitilecek parametre sayısı tümüyle karşılaştığımızda ne kadar küçük .

trainable params: 393,216 || all params: 559,607,808 || trainable%: 0.07026635339584111
None


In [14]:
import os
working_dir='./'

# burada bir çıktı dizini ayarlanmış oldu

output_directory = os.path.join(working_dir, 'peft_lab_outpus')

In [15]:
## Training Args'da epoch sayısını ve çıktı dizininini ayrıca learning rate 'i belirleriz

import transformers
from transformers import TrainingArguments, Trainer
training_args = TrainingArguments(
    output_dir = output_directory,
    auto_find_batch_size = True, #  veriyle en uyumlu olacak batch büyüklüğünü ayarlar.
    learning_rate = 3e-2,
    num_train_epochs = 2,
    use_cpu = True
)

In [16]:
# Artık modeli eğtimek için tüm argumanlarımızı oluşşturduk.
"""
* PEFT Model
* Training_args
* Dataset
* datacollator  model eğitimi için veri setinden alınan örnekleri hazırlamak ve düzenlemek için kullanılır

"""

#  5-10 dakika sürebilir.

trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset = train_sample,
    data_collator = transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)

)

trainer.train()

Step,Training Loss


TrainOutput(global_step=14, training_loss=2.725315366472517, metrics={'train_runtime': 396.26, 'train_samples_per_second': 0.252, 'train_steps_per_second': 0.035, 'total_flos': 20073085894656.0, 'train_loss': 2.725315366472517, 'epoch': 2.0})

In [17]:
# modeli kaydetme

peft_model_path = os.path.join(output_directory,f"lora_model_yeni")

trainer.model.save_pretrained(peft_model_path)

In [18]:
# modeli yükleme

loaded_model = PeftModel.from_pretrained(foundation_model,peft_model_path,is_trainable=False)

In [24]:
input_sentence = tokenizer("ı want you to be a motivational coach.", return_tensors="pt")
foundational_outputs_sentence = get_output(loaded_model,input_sentence, max_new_tokens=50)
print(tokenizer.batch_decode(foundational_outputs_sentence, skip_special_tokens=True))

['ı want you to be a motivational coach. I will provide the information about your goals and lifestyle style, as well as:  My first request is "I need help improving my life" "I\'ve been struggling with depression lately so it\'s hard for me not being able focus on anything else than workouts!']


In [ ]:
###    eğitilmiş ve eğitilmemiş model arasındaki fark çok büyük.